In [23]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import math

In [29]:
def normalise(img):
    img = np.float32(img)
    img -= np.min(img)
    img /= np.max(img)
    img = img*2 - 1
    return img

def norm_uint8(img):
    return np.uint8((normalise(img)+1)*127.5)

def resize_to_maxsidelen(img, size):
    size_max = np.max(img.shape[:2])
    r = size/size_max
    img = cv2.resize(img, (int(round(img.shape[1]*r)), int(round(img.shape[0]*r))))
    return img

def resize_to_minsidelen(img, size):
    size_min = np.min(img.shape[:2])
    r = size/size_min
    img = cv2.resize(img, (int(round(img.shape[1]*r)), int(round(img.shape[0]*r))))
    return img

def zoom_augmentations(img, target_size=512, num=10, max_offset=40):
    augs = []
    for i in range(0, max_offset, max_offset//num):
        zoomed = resize_to_minsidelen(img, target_size + i)
        shift_x = (zoomed.shape[0] - target_size) // 2
        shift_y = (zoomed.shape[1] - target_size) // 2
        crop = zoomed[shift_x : shift_x + target_size, shift_y : shift_y + target_size]
        augs.append(crop)
    return augs


def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result


def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

def get_all_rotations(img):
    rots = []
    for angle in range(0, 360, 45):
        image_rotated = rotate_image(img, angle)
        image_rotated_cropped = crop_around_center(
            image_rotated,
            *largest_rotated_rect(
                img.shape[1],
                img.shape[0],
                math.radians(angle)))
        rots.append(image_rotated_cropped)
    return rots

def rot(img, angle):
    image_rotated = rotate_image(img, angle)
    image_rotated_cropped = crop_around_center(
        image_rotated,
        *largest_rotated_rect(
            img.shape[1],
            img.shape[0],
            math.radians(angle)))
    return image_rotated_cropped

In [31]:
# export 45* rotations
dir = "/path/"
out_dir = "/path/"
imgs_paths = os.listdir(dir)

for filename in imgs_paths:
    img = cv2.imread(dir + filename)
    rotated = rot(img, 45)
    cv2.imwrite(out_dir+filename[:-4]+"_45.jpg", rotated)
    rotated = rot(img, 315)
    cv2.imwrite(out_dir+filename[:-4]+"_315.jpg", rotated)

In [31]:
# apply zoom augs
dir = "/path/"
out_dir = "/path/"
imgs_paths = os.listdir(dir)

for filename in imgs_paths:
    augs = zoom_augmentations(img)
    for i, img in enumerate(augs):
        cv2.imwrite(out_dir+filename[:-4]+'_'+str(i)+".jpg")

True